In [1]:
%pip install -qU langchain langchain_openai langchain-core langgraph langchain-community psycopg[binary,pool]==3.2.6 langgraph-checkpoint-postgres langchain-elasticsearch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 973.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.3/906.3 kB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 k

In [2]:
import os
with open("/content/api_key.txt") as archivo:
  apikey = archivo.read()
os.environ["OPENAI_API_KEY"] = apikey

with open("/content/postgrest.txt") as archivo:
  uribd = archivo.read()

In [3]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage
from langchain_elasticsearch import ElasticsearchStore
from psycopg_pool import ConnectionPool
from langgraph.checkpoint.postgres import PostgresSaver
from langgraph.prebuilt import create_react_agent

In [4]:
from langchain_core.output_parsers import StrOutputParser
from langchain_community.utilities import SQLDatabase
from langchain.agents import Tool

In [5]:
db = SQLDatabase.from_uri(uribd)

def get_schema() -> str:
    return db.get_table_info()

prompt_sql = ChatPromptTemplate.from_template("""
Dado el siguiente esquema de la base de datos y una pregunta, genera solamente una consulta SQL válida y completa.

Si la pregunta menciona un código de pedido, genera una consulta que:
Incluya el estado del pedido y la fecha de entrega.
Traiga los productos relacionados a ese pedido.
No uses comillas triples, bloques de código ni explicaciones.

Esquema de la base de datos:
{schema}

Pregunta del usuario:
{question}

Consulta SQL (sin formato adicional):
""")

sql_chain = prompt_sql | ChatOpenAI(model="gpt-4.1-2025-04-14").bind(stop=["\nSQLResult"]) | StrOutputParser()

def generar_sql(schema: str, question: str) -> str:
    return sql_chain.invoke({"schema": schema, "question": question})

def run_query(query: str) -> str:
    return db.run(query)

prompt_respuesta = ChatPromptTemplate.from_template("""
Basado en el esquema, pregunta, SQL y resultado, responde en lenguaje natural.

Esquema:
{schema}

Pregunta:
{question}

SQL:
{query}

Resultado:
{response}

Respuesta natural:
""")

sqlnatural_chain = prompt_respuesta | ChatOpenAI(model="gpt-4.1-2025-04-14") | StrOutputParser()

def generar_respuesta(schema: str, question: str, query: str, response: str) -> str:
    return sqlnatural_chain.invoke({
        "schema": schema,
        "question": question,
        "query": query,
        "response": response
    })

In [6]:
def consultar_pedido(pregunta: str) -> str:
    schema = get_schema()
    sql = generar_sql(schema, pregunta)
    resultado = run_query(sql)
    return generar_respuesta(schema, pregunta, sql, resultado)

tool_consulta_pedido = Tool(
    name="consulta_pedido",
    func=consultar_pedido,
    description="Usa esta herramienta para obtener informacion detallada de un pedido."
)

In [7]:
db_vector = ElasticsearchStore(
    es_url="http://34.10.99.202:9200",
    es_user="elastic",
    es_password="qtPIwPpL7UamZQqjuqlm",
    index_name="lg-politicas",
    embedding=OpenAIEmbeddings())

retriever = db_vector.as_retriever()

tool_rag =retriever.as_tool(
        name="politicas",
        description="Consulta solo politicas de reclamos y reembolsos para determinar si procede o no el reembolso en un caso específico.",
    )

/tmp/ipython-input-7-2314541577.py:10: LangChainBetaWarning: This API is in beta and may change in the future.
  tool_rag =retriever.as_tool(


In [8]:
prompt = ChatPromptTemplate.from_messages([
    ("system", """
Eres un asistente experto en seguimiento de pedidos y reclamos de una tienda de computadoras. Atiendes a los clientes de forma clara, empática y breve, como un amigo experto en tecnología.

FLUJO DE ATENCIÓN:

1. **Saludo e identificación**:
   - Saluda cordialmente.
   - Si el cliente no da un código de pedido, solicítalo antes de continuar.

2. **Verificación del pedido**:
   - Si el pedido está en curso, solo indícale que puede consultar el estado en la web.

3. **Detección del problema**:
   - Muestra al cliente los productos del pedido.
   - Pregunta cuál(es) tienen problemas y qué tipo: **Faltante**, **Maltratado** o **Erróneo**.
   - Solo continúas si el problema es uno de esos tres.

4. **Revisión y resolución**:
   - Verifica si el producto figura en el pedido.
   - Usa la herramienta para consultar las políticas oficiales.
   - Si tienes la fecha de entrega, calcula tú si el reclamo está dentro del plazo de 3 días hábiles (no preguntes al cliente).
   - Si todo aplica, calcula el monto exacto del reembolso y comunícalo de forma directa:
     - Menciona el porcentaje y el monto final.
     - Indica que el reembolso será en **créditos de la app**, a menos que el cliente prefiera su tarjeta.
   - **No repitas el texto de la política al cliente. Solo aplica la decisión con lenguaje natural.**

5. **Cierre**:
   - Agradece al cliente y pregunta si necesita algo más.

REGLAS CLAVE:

- Usa solo las herramientas disponibles.
- No inventes reglas, productos ni estados.
- No confirmes reembolsos si el producto no figura, el problema no es válido o el plazo expiró.
- **No muestres ni expliques el texto exacto de las políticas al cliente. Aplica el criterio directamente.**
- El reembolso por defecto se hace en **créditos de la app**. Solo si el cliente los rechaza, se ofrece devolución a tarjeta (5 a 10 días hábiles).
"""),
    ("human", "{messages}"),
])

In [ ]:
connection_kwargs = {
    "autocommit": True,
    "prepare_threshold": 0,
}

with ConnectionPool(
    conninfo=uribd,
    max_size=20,
    kwargs=connection_kwargs,
) as pool:
    checkpointer = PostgresSaver(pool)
    checkpointer.setup()

In [9]:
connection_kwargs = {
    "autocommit": True,
    "prepare_threshold": 0,
}

with ConnectionPool(
    conninfo=uribd,
    max_size=20,
    kwargs=connection_kwargs,
) as pool:
    checkpointer = PostgresSaver(pool)

    # Modelo
    model = ChatOpenAI(model="gpt-4.1-2025-04-14")

    # Toolkit de herramientas
    toolkit = [tool_rag, tool_consulta_pedido]

    # Crear agente con prompt y memoria
    agent_executor = create_react_agent(
        model=model,
        tools=toolkit,
        checkpointer=checkpointer,
        prompt=prompt
    )

    config = {"configurable": {"thread_id": "mc0035"}}

    # Ejemplo de ejecución
    for step in agent_executor.stream(
        {"messages": [HumanMessage(content="Hola")]},
        config=config,
        stream_mode="values",
    ):
        step["messages"][-1].pretty_print()


================================ Human Message =================================

Hola
================================== Ai Message ==================================

¡Hola! ¿En qué puedo ayudarte hoy? Si tienes un pedido y necesitas hacer seguimiento o tienes un reclamo, por favor dime tu código de pedido para poder revisarlo.
